## 14. 순환신경망 ##
### 14.1 순환뉴런 ###

![RNN1](images/rnn1.png)

<center><b style="font-size : 1.2em">순환 뉴런 & 타임 스텝으로 펼친 뉴런</b></center>

![RNN2](images/rnn2.png)

<center><b style="font-size : 1.2em">순환 뉴런의 층 & 타입 스텝으로 펼쳐진 층</b></center>

각 순환 뉴런은 두 개의 가중치 $\mathbf{w}_{x}$ 와 $\mathbf{w}_{y}$ 를 가지는데, $\mathbf{w}_{x}$ 는 $\mathbf{x}_{t}$ 를 위한 것이고 $\mathbf{w}_{y}$는 이전 타임 스텝의 출력 $\mathbf{y}_{t-1}$을 위한 것이다. 이것을 순환 층(layer) 전체로 생각하면 가중치 벡터 $\mathbf{w}_{x}$ 와 $\mathbf{w}_{y}$ 를 행렬 $\mathbf{W}_{x}$ 와 $\mathbf{W}_{y}$ 로 나타낼 수 있으며 다음의 식과 같이 표현할 수 있다.

$$
\mathbf{y}_{t} = \phi \left( \mathbf{W}_{x}^{T}\cdot\mathbf{x}_{t} + \mathbf{W}_{y}^{T} \cdot\mathbf{y}_{t-1} + \mathbf{b} \right)
$$

<br>

그리고 타임 스텝 $t$ 에서의 미니배치(mini-batch)의 입력을 행렬 $\mathbf{X}_{t}$ 로 나타내어 아래와 같이 순환 층의 출력을 한번에 계산할 수 있다.

$$
\mathbf{Y}_{t}= \phi \left( \mathbf{X}_{t} \cdot \mathbf{W}_{x} + \mathbf{Y}_{t-1} \cdot \mathbf{W}_{y} + \mathbf{b} \right)
$$

$$
= \phi \left( \begin{bmatrix} \mathbf{X}_{t} \mathbf{Y}_{t-1} \end{bmatrix}\begin{bmatrix} \mathbf{W}_{x} \\ \mathbf{W}_{y} \end{bmatrix} + \mathbf{b} \right)
$$

$$
여기서 W = \begin{bmatrix} \mathbf{W}_{x} \\ \mathbf{W}_{y} \end{bmatrix}
$$



$\phi$는 활성화 함수이다, RNN에서의 활성화 함수로는 $\tanh$ 가 주로 사용된다


<br/>

<ul>
<li>$\mathbf{Y}_{t}$ : 타임 스텝 $t$에서 미니배치에 있는 각 샘플(미니배치)에 대한 순환 층의 출력이며, <b style="color:red; font-size:1.1em">$m \times n_{\text{neurons}}$</b> 행렬($m$은 미니배치, $n_{\text{neurons}}$은 뉴런 수), <b style="background-color:#00C500">각 뉴런별로 출력값이 나오기때문에 샘플의 수 X 뉴런의 수</b></li>
<li>$\mathbf{X}_{t}$ : 모든 샘플의 입력값을 담고 있는 <b style="color:red; font-size:1.1em">$m \times n_{\text{inputs}}$</b> 행렬 ($n_{\text{inputs}}$은 입력 특성 수)</li>
<li>$\mathbf{W}_{x}$ : 현재 타임 스텝 $t$의 입력에 대한 가중치를 담고 있는 <b style="color:red; font-size:1.1em">$n_{\text{inputs}} \times n_{\text{neurons}}$</b> 행렬</li>
<li>$\mathbf{W}_{y}$ : 이전 타임 스텝 $t-1$ 의 출력에 대한 가중치를 담고 있는 <b style="color:red; font-size:1.1em">$n_{\text{neurons}} \times n_{\text{neurons}}$</b> 행렬</li>
<li>$\mathbf{b}$ : 각 뉴런의 편향(bias)을 담고 있는 $n_{\text{neurons}}$ 크기의 벡터</li>
<li>$\begin{bmatrix} \mathbf{X}_{t} \mathbf{Y}_{t-1} \end{bmatrix}$는 $\mathbf{X}_{t}\mathbf{Y}_{t-1}$을 수평적으로 연결한다는 것을 의미</li>
</ul>
위의 식에서 $\mathbf{Y}_{t}$ 는 $\mathbf{X}_t$ 와 $\mathbf{Y}_{t-1}$의 함수이므로, 타임 스텝 $t=0$ 에서부터 모든 입력에 대한 함수가 된다. 첫 번째 타임 스텝인 $t=0$ 에서는 이전의 출력이 없기 때문에 일반적으로 $0$으로 초기화 한다.

### 입력과 출력 시퀀스 ###

![RNN4](images/rnn4.png)


<ul>
    <li><p><b>Vector to Sequence</b> : 첫 번째 타임 스텝에서 단 하나의 입력만 네트워크에 주입하고(다른 모든 타임 스텝에서는 0 주입) 출력시퀀스 연결, 예)이미지 캡션 출력</p></li>
    <li><p><b>Sequence to Vector</b> : 입력시퀀스를 네트워크에 주입하고, 마지막을 제외한 모든 출력을 무시, 예)감성점수 출력</p></li>
    <li><p><b>Sequence to Sequence</b> : 입력시퀀스를 받아 출력시퀀스를 만듬, 예)주식가격 예측</p></li>
    <li><p><b>Delayed Sequence to Sequence</b> : 시퀀스-투-벡터(인코더) 뒤에 벡터-투-시퀀스(디코더) 연결, 예)문장 번역</p></li>
</ul>

### 14.2 텐서플로로 기본 RNN 구성하기 ###

In [17]:
import tensorflow as tf

## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [18]:
tf.reset_default_graph()

n_inputs = 3 #입력 특성의 개수
n_neurons = 5 #순환 뉴런층 안에 있는 뉴런의 개수

X0 = tf.placeholder(tf.float32, [None, n_inputs]) #step0에서 입력
X1 = tf.placeholder(tf.float32, [None, n_inputs]) #step1에서 입력

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32)) #입력을 위한 가중치
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32)) #출력을 위한 가중치
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32)) #편향

Y0 = tf.tanh(tf.matmul(X0, Wx) + b) #step0에서 출력
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b) #step1에서 출력

init = tf.global_variables_initializer()

In [19]:
import numpy as np

#미니배치            샘플0      샘플1      샘플2      샘플3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0 ,1]]) #step0에서 입력
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) #step1d에서 입력

with tf.Session() as sess:
    sess.run(init)
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    
print('Y0_val:{}\n{}'.format(Y0_val.shape, Y0_val))  # shape: (4, 5) → (샘플 개수, 뉴런 개수)
print('Y1_val:{}\n{}'.format(Y1_val.shape, Y1_val))  # shape: (4, 5) → (샘플 개수, 뉴런 개수)

Y0_val:(4, 5)
[[ 0.97126496 -0.98614025 -0.9352247  -0.24996306  0.9982884 ]
 [ 0.4023137  -0.9999944  -0.9999988   0.30736047  0.9999865 ]
 [-0.85143423 -1.         -1.          0.71170443  0.9999999 ]
 [-1.         -0.9993603  -0.9999997   0.99999934 -1.        ]]
Y1_val:(4, 5)
[[-0.9999995  -0.99999976 -1.          0.71982044  0.9903299 ]
 [-0.74468136  0.03505388  0.8692696  -0.87766755 -0.82537484]
 [-0.99999493 -0.99999875 -0.9999983  -0.7216424   0.01635024]
 [-0.9997643  -0.99988526 -0.99997747  0.98211133 -0.5999049 ]]


In [20]:
#위와 동일한 코드

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

# BasicRNNCell
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static_rnn()
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1],
                                           dtype=tf.float32) #셀을 연결하여 펼쳐진 RNN네트워크를 만듬
Y0, Y1 = output_seqs #각 타임스텝에서의 출력 텐서를 담고있는 리스트, 네트워크의 최종 상태를 담고있는 텐서

#미니배치
# t = 0
X0_batch = np.array([[0, 1, 2],  # sample 0
                     [3, 4, 5],  # sample 1
                     [6, 7, 8],  # sample 2
                     [9, 0, 1]]) # sample 3
# t = 1
X1_batch = np.array([[9, 8, 7], 
                     [3, 4, 5], 
                     [6, 5, 4], 
                     [3, 2, 1]])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    
print('Y0_val:{}\n{}'.format(Y0_val.shape, Y0_val))  # shape: (4, 5) → (샘플 개수, 뉴런 개수)
print('Y1_val:{}\n{}'.format(Y1_val.shape, Y1_val))  # shape: (4, 5) → (샘플 개수, 뉴런 개수)

Y0_val:(4, 5)
[[ 0.6665392  -0.27402034 -0.14346613  0.7007153   0.5232706 ]
 [ 0.8718256  -0.96863294  0.9426004   0.3721035   0.98638576]
 [ 0.9542055  -0.9991093   0.99869287 -0.08676071  0.99969983]
 [-0.99974024 -0.9242235   0.9999719  -0.99957     0.99999136]]
Y1_val:(4, 5)
[[ 0.6852712  -0.99996364  0.99999326 -0.984032    0.9999805 ]
 [ 0.94351286 -0.98605835  0.99266964 -0.08152092  0.9966243 ]
 [ 0.6018127  -0.9986716   0.99989456 -0.9759625   0.9991661 ]
 [ 0.54286885 -0.9488012   0.986592   -0.9866974   0.75857   ]]


In [34]:
#출력텐서 간단하게 만들기

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) #타임스텝별로 입력을 만드는 대신 타입스템만큼의 열을 만듬
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2])) #입력데이터의 열을 바꾸고 하나씩 꺼냄

# BasicRNNCell
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static_rnn()
output_seqs, states = tf.nn.static_rnn(basic_cell, X_seqs, dtype=tf.float32) #셀을 연결하여 펼쳐진 RNN네트워크를 만듬
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0 ,2]) #모든 출력을 하나의 텐서로 합침

#미니배치
X_batch = np.array([[[0, 1, 2], [9, 8, 7]], # sample 0
                     [[3, 4, 5], [3, 4, 5]], # sample 1
                     [[6, 7, 8], [6, 5, 4]], # sample 2
                     [[9, 0, 1], [3, 2, 1]]]) # sample 3

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    
print(outputs_val) # shape: (4, 2, 5) → (샘플 개수, 타임스텝 개수, 뉴런 개수)

[[[-0.1105443  -0.50030005 -0.35142988  0.9068353  -0.3320599 ]
  [-0.9999975  -0.96750146 -0.6956085   0.83011717 -1.        ]]

 [[-0.9874154  -0.8828229  -0.7572333   0.9866959  -0.99804294]
  [-0.987381   -0.93349     0.52412933  0.97658026 -0.99984115]]

 [[-0.99989986 -0.9770097  -0.92350227  0.998166   -0.9999962 ]
  [-0.99985003 -0.91482174  0.6592921   0.27506992 -0.9999988 ]]

 [[-0.99841654 -0.8882804  -0.9894495  -0.99995464 -0.9999384 ]
  [-0.998729   -0.82644653  0.79904616  0.02509189 -0.99954045]]]


### static_rnn()의 문제점

<span>static_rnn() 함수의 문제는 타임 스텝마다 하나의 셀을 그래프에 추가하기 때문에, 만약 타임 스텝이 많아질 경우 그래프가 매우 복잡해진다는 것이다.</span>
<br>
<span>쉽게 말하면, for문과 같이 반복문을 쓰지않고 동일한 셀을 타임 스텝별로 만드는 것이라 할 수 있다.</span>
<br>
<span>이럴 경우 타임 스텝이 많아서 그래프가 커지게되면 역전파(backprop)시 메모리 부족(OOM, Out-Of-Memory)에러가 발생할 수 있다.</span>
    
<span>이러한 문제를 해결할 수 있는 방법으로는 tf.nn.dynamic_rnn()이 있다.</span>

## 2.2 동적으로 타임 스텝 펼치기

<p style="font-size:1.1em">tf.nn.dynamic_rnn() 함수는 적절한 타임 스텝에 걸쳐 셀을 실행하기 위해 tf.while_loop() 연산을 사용.</p>
<p style="font-size:0.9em;color:blue">tf.while_loop() 연산을 사용하면 타임 스텝의 크기에 상관없이 하나의 행렬곱 연산 tf.matmul()만 생성함</p>
<br>
<p style="font-size:1.1em">GPU메모리에서 CPU메모리로 바꾸어 역전파 시에 OOM(Out-Of-Memory)에러를 피하기 위해 tf.nn.dynamic_rnn(swap_memory=True) 설정</p>
<p style="font-size:1.1em">이 함수는 각 타임 스텝의 모든 입력에 대해 텐서 하나를 받고([None, n_steps, n_inputs]), 타임 스텝마다의 모든 출력을 하나의 텐서([None, n_steps, n_neurons])로 반환</p>
<p style="font-size:0.9em;color:blue">tf.stack, tf.unstack, tf.transpose 사용할 필요 없음!!</p>

In [40]:
#출력텐서 간단하게 만들기

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) #타임스텝별로 입력을 만드는 대신 타입스템만큼의 열을 만듬
#tf.stack 사용할 필요없음
#X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2])) #입력데이터의 열을 바꾸고 하나씩 꺼냄

# BasicRNNCell
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static_rnn()
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, swap_memory=True) #OOM시 메인 메모리 사용
#tf.stack, tftranspose 사용할 필요없음
#outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0 ,2]) #모든 출력을 하나의 텐서로 합침

#미니배치
X_batch = np.array([[[0, 1, 2], [9, 8, 7]], # sample 0
                     [[3, 4, 5], [3, 4, 5]], # sample 1
                     [[6, 7, 8], [6, 5, 4]], # sample 2
                     [[9, 0, 1], [3, 2, 1]]]) # sample 3

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    
print(outputs_val) # shape: (4, 2, 5) → (샘플 개수, 타임스텝 개수, 뉴런 개수)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
[[[ 0.85334516  0.8141438   0.313379   -0.69611907  0.28773725]
  [ 0.99999994  1.          0.9913502  -0.8543494   0.98387647]]

 [[ 0.99980074  0.9998129   0.86700404 -0.8867808   0.8650835 ]
  [ 0.99959165  0.999959    0.9114392  -0.97390735  0.7209359 ]]

 [[ 0.99999976  0.9999998   0.98077375 -0.9605962   0.98125607]
  [ 0.99991673  0.9999967   0.95923656 -0.76816344  0.86428374]]

 [[ 0.9999454   0.9983292  -0.8371234   0.99967825 -0.3139772 ]
  [ 0.90131366  0.9690799  -0.28888935 -0.15881321  0.7139215 ]]]


## 2.3 가변 길이 입력 시퀀스 다루기

문장같은 가변 길이 입력 시퀀스를 다루기 위해서는 <b style="background-color:#0cf">tf.nn.dynamic_rnn(sequence_length={입력 시퀀스 길이}),</b> 또는<b style="background-color:#0ca">tf.nn.static_rnn(sequence_length={입력 시퀀스 길이})</b>를 설정해야 함

In [45]:
#출력텐서 간단하게 만들기

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) #타임스텝별로 입력을 만드는 대신 타입스템만큼의 열을 만듬
seq_length = tf.placeholder(shape=[None], dtype=tf.float32)

# BasicRNNCell
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static_rnn()
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, swap_memory=True, sequence_length=seq_length) #가변길이 설정

#미니배치
X_batch = np.array([[[0, 1, 2], [9, 8, 7]], # sample 0
                     [[3, 4, 5], [0, 0, 0]], # sample 1, 모자란 부분은 0으로 채워서 길이를 맞춰줌(0으로 패딩시킴)
                     [[6, 7, 8], [6, 5, 4]], # sample 2
                     [[9, 0, 1], [3, 2, 1]]]) # sample 3
seq_length_batch = [2, 1, 2, 2] #입력데이터의 길이

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

print("-------------------- outputs_val --------------------")
print(outputs_val) # shape: (4, 2, 5) → (샘플 개수, 타임스텝 개수, 뉴런 개수)

print()

print("-------------------- states_val --------------------")
print(states_val) # shape: (4,5) → (샘플 개수, 뉴런 개수), 0벡터가 제외된 각 셀의 마지막 상태를 담음

-------------------- outputs_val --------------------
[[[ 0.1711922  -0.6476346   0.39549306  0.03427123  0.35730192]
  [ 0.9999266  -0.99998343  0.16134876 -0.99387264  0.9853087 ]]

 [[ 0.95986235 -0.9952373   0.36473402 -0.6991825   0.88534856]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.9988153  -0.9999467   0.33315763 -0.9431344   0.98450124]
  [ 0.979026   -0.9992811   0.27179083 -0.9526911   0.7239816 ]]

 [[ 0.9736584  -0.99963105  0.80333865 -0.9926766  -0.85368717]
  [ 0.91740143 -0.95162207  0.2917505  -0.06991738  0.03283571]]]

-------------------- states_val --------------------
[[ 0.9999266  -0.99998343  0.16134876 -0.99387264  0.9853087 ]
 [ 0.95986235 -0.9952373   0.36473402 -0.6991825   0.88534856]
 [ 0.979026   -0.9992811   0.27179083 -0.9526911   0.7239816 ]
 [ 0.91740143 -0.95162207  0.2917505  -0.06991738  0.03283571]]
